## Описание данных
    
### Имеются следующие данные о транзакциях в период с 01.12.2010 по 12.09.2011:
- **InvoiceNo** — номер транзакции
- **StockCode** — код товара
- **Description** — описание товара
- **Quantity** — количество единиц товара, добавленных в заказ
- **InvoiceDate** — дата транзакции 
- **UnitPrice** — цена за единицу товара
- **CustomerID** — id клиента
- **Country** — страна, где проживает клиент

In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [2]:
retail_df = pd.read_csv('retail_data.zip', compression='zip', encoding='ISO-8859-1')
retail_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [3]:
retail_df.shape

(541909, 8)

In [4]:
retail_df.dtypes

InvoiceNo       object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
UnitPrice      float64
CustomerID     float64
Country         object
dtype: object

In [5]:
missing = retail_df.isnull().sum()
missing[missing > 0]

Description      1454
CustomerID     135080
dtype: int64

# <font color='Navy'>Задачи:</font>

#### 1. Проверьте, встречаются ли в данных повторяющиеся наблюдения, и в качестве ответа укажите их количество. Если они есть, то удалите их из retail

In [6]:
print(f' Количество дублирующихся значений в датафрейме - {retail_df.duplicated().sum()}')

 Количество дублирующихся значений в датафрейме - 5268


In [7]:
# Удалим все дубликаты из датафрейма
retail_df.drop_duplicates(inplace=True)
retail_df.duplicated().sum()

0

#### 2. Данные содержат в себе записи как об успешных транзакциях, так и об отмененных. Если пользователь отменил заказ, в начале номера транзакции (InvoiceNo) ставится C (canceled). <font color='#1F618D'> Сколько всего заказов отменили пользователи?</font>

In [8]:
canceled = retail_df.InvoiceNo.str.startswith('C').sum()
print(f'Количество отмененных пользователями заказов - {canceled}')

Количество отмененных пользователями заказов - 9251


#### 3. Теперь отфильтруйте данные и оставьте в retail только те заказы, где Quantity > 0. В качестве ответа укажите число оставшихся строк.

In [9]:
retail = retail_df.query('Quantity > 0')
retail.shape

(526054, 8)

#### 4. Посчитайте число заказов для каждого пользователя (CustomerID) из Германии (Germany). Оставьте только тех, кто совершил более N транзакций (InvoiceNo), где N – 80% процентиль. Запишите полученные id пользователей в germany_top (не весь датафрейм, только id).
```
*Примечание. Для каждого заказа в данных может встречаться более 1 строки.
```


In [10]:
# Отберем всех клиентов из Германии, сгруппируем по CustomerID и посчитаем кол-во уникальных значений
german_buyers = retail \
            .query('Country == "Germany"') \
            .groupby('CustomerID', as_index=False) \
            .agg({'InvoiceNo': 'nunique'}) \
            .rename(columns={'InvoiceNo': 'OrdersNo'})

german_buyers.head()

,CustomerID,OrdersNo
0,12426.0,1
1,12427.0,3
2,12468.0,2
3,12471.0,30
4,12472.0,7


In [11]:
# Создадим переменную с 80-м процентилем и запишем в переменную germany_top id клиентов, которые совершили больше всего покупок
top_20 = german_buyers.OrdersNo.quantile(q=0.8)
germany_top = german_buyers.query('OrdersNo > @top_20')['CustomerID']
germany_top[:5]

3     12471.0
6     12474.0
8     12476.0
12    12481.0
16    12500.0
Name: CustomerID, dtype: float64

#### 5. Используя объект с id пользователей (germany_top), полученный на предыдущем шаге, отфильтруйте наблюдения и оставьте в данных записи только по интересующим нас юзерам. Результирующий датафрейм запишите в top_retail_germany.

In [12]:
top_retail_germany = retail.query('CustomerID in @germany_top')
top_retail_germany.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
1109,536527,22809,SET OF 6 T-LIGHTS SANTA,6,12/1/2010 13:04,2.95,12662.0,Germany
1110,536527,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,6,12/1/2010 13:04,2.55,12662.0,Germany
1111,536527,84945,MULTI COLOUR SILVER T-LIGHT HOLDER,12,12/1/2010 13:04,0.85,12662.0,Germany
1112,536527,22242,5 HOOK HANGER MAGIC TOADSTOOL,12,12/1/2010 13:04,1.65,12662.0,Germany
1113,536527,22244,3 HOOK HANGER MAGIC GARDEN,12,12/1/2010 13:04,1.95,12662.0,Germany


#### 6. Сгруппируйте top_retail_germany по коду товара (StockCode). Какой из продуктов добавляли в корзину чаще всего, кроме POST? 

*Note: одним заказом считается единовременная покупка любого количества товара, т.е. без учета Quantity.*

In [13]:
top_retail_germany \
                .groupby('StockCode', as_index=False) \
                .agg({'InvoiceDate': 'nunique'}) \
                .rename(columns={'InvoiceDate': 'Number'}) \
                .sort_values('Number', ascending=False) \
                .iloc[1]

StockCode    22326
Number          52
Name: 409, dtype: object

#### 7. Вернемся к анализу полного датасета retail. Создайте колонку Revenue с суммой покупки, используя колонки Quantity и UnitPrice.

In [14]:
retail['Revenue'] = retail['Quantity'] * retail['UnitPrice']
retail.head(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,12/1/2010 8:26,7.65,17850.0,United Kingdom,15.30
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,12/1/2010 8:26,4.25,17850.0,United Kingdom,25.50
7,536366,22633,HAND WARMER UNION JACK,6,12/1/2010 8:28,1.85,17850.0,United Kingdom,11.10
8,536366,22632,HAND WARMER RED POLKA DOT,6,12/1/2010 8:28,1.85,17850.0,United Kingdom,11.10
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,12/1/2010 8:34,1.69,13047.0,United Kingdom,54.08


#### 8. Для каждой транзакции (InvoiceNo), посчитайте финальную сумму заказа. В качестве ответа укажите топ-5 (InvoiceNo) по сумме заказа (через запятую с пробелом и в порядке убывания TotalRevenue).

In [15]:
top_orders = retail \
                .groupby('InvoiceNo', as_index=False) \
                .agg({'Revenue': 'sum'}) \
                .rename(columns={'Revenue': 'TotalRevenue'}) \
                .sort_values('TotalRevenue', ascending=False)

top_orders.head()

,InvoiceNo,TotalRevenue
20689,581483,168469.60
2202,541431,77183.60
17582,574941,52940.94
18251,576365,50653.91
9034,556444,38970.00


In [16]:
print(top_orders.InvoiceNo[:5].str.cat(sep=', '))

581483, 541431, 574941, 576365, 556444
